### Birzeit University

### Department of Electrical & Computer Engineering

### First Semester, 2023/2024

### ENCS5343 Computer Vision

### Assignment#2 CBIR Ststem

### Mohammad AbuJaber - 1190298


# Introduction


Content-Based Image Retrieval (CBIR) is a computer vision and information retrieval technique that uses visual features of images to search and retrieve them from a database. Unlike traditional methods that rely on metadata or textual descriptions, CBIR focuses on the visual features of images, such as color, texture, shape, and spatial arrangements. CBIR systems use an image query to search for similar images in a database by comparing the visual content of the query image with the features extracted from the stored images. This involves feature extraction techniques to represent visual content numerically and similarity measures to quantify the resemblance between images. CBIR has various applications, including medical imaging, multimedia retrieval, e-commerce, and law enforcement. It offers advantages such as no need for keywords, more accurate results, faster searching, and no subjective interpretation. However, CBIR requires access to a large and well-indexed database of images and can be computationally expensive. Despite these limitations, CBIR is a powerful technology that is constantly evolving, making it an essential tool for anyone working with images.


Color histograms and color moments are two essential tools for analyzing and summarizing the color distribution within an image. Both are based on probability theory and offer complementary insights for applications like image retrieval, segmentation, and classification.

### Color histogram

Color histograms are graphical representations of the distribution of colors within an image, dividing the color space into discrete bins and counting the number of pixels falling into each bin. They are simple to compute and visually intuitive, but are sensitive to illumination changes and prone to quantization errors.

### Color moments

Color moments are statistical measures that capture properties of the color distribution in an image, calculated from the mean, variance, and higher-order moments of the pixel color intensities. They are based on statistical moment theory and provide quantitative descriptors of the image's color distribution. They are robust to illumination changes, insensitive to quantization errors, and capture spatial information through higher-order moments.

The choice between color histograms and color moments depends on the specific application and desired information. Histograms are efficient for gross color trends and quick comparisons, while moments offer deeper statistical insights and robustness to lighting changes. Combining both approaches can provide a comprehensive understanding of an image's color content.


### Dataset

A modified version of ["corel-images"](https://www.kaggle.com/datasets/elkamel/corel-images) datasert consists of two main directories, "horses_test" and "training_set," organized into subdirectories corresponding to various image categories. The "horses_test" directory contains images specifically chosen for testing or querying the Content-Based Image Retrieval (CBIR) system. The "training_set" directory includes images from 10 distinct categories, such as beaches, buses, dinosaurs, elephants, flowers, foods, horses, monuments, mountains_and_snow, and people_and_villages_in_Africa. Each subdirectory contains images relevant to its category, forming a comprehensive training set for the CBIR system. This variety enables the CBIR system to learn and recognize diverse visual features associated with different image categories, contributing to its ability to retrieve visually similar images based on color histograms.

For the modified versoion of dataset ["click here"](https://drive.google.com/drive/folders/1F7wiYMAxT43enCSYc3THVgDjLoIAeoOD?usp=sharing).


> ### **Task 1:**
>
> Build the CBIR system: Design and implement a system architecture for image retrieval using color features. Develop functionalities for loading images, extracting features, computing distances, and ranking results.


The following code shows an example implementation of CBIR System.


### System Setup (CBIRSystem Class)

- **`__init__` (Initialization):** The class takes a dataset path as input and initializes empty lists for image paths and features.
- **`load_images`:** This method traverses the dataset path and collects paths of images with specific file extensions (png, jpg, jpeg).
- **`extract_features`:** For each image in the dataset, it extracts color histograms using the calculate_histogram method and flattens them for feature representation.
- **`calculate_histogram`:** This method computes a 3D color histogram for an image using the OpenCV function cv2.calcHist. The histogram is then normalized.
- **`search_similar_images`:** Given a query image path, this method calculates the color histogram for the query image and compares it with the histograms of all images in the dataset using Euclidean distance. The top-k similar images are returned based on the smallest Euclidean distances.
- **`plot_results`:** This method visualizes the query image and the top-k similar images in a 2x5 grid using Matplotlib.


In [ ]:
import os
import cv2
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt


class CBIRSystem:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.image_paths = []
        self.features = []

    def load_images(self):
        for root, dirs, files in os.walk(self.dataset_path):
            for file in files:
                if file.lower().endswith((".png", ".jpg", ".jpeg")):
                    image_path = os.path.join(root, file)
                    self.image_paths.append(image_path)

    def extract_features(self):
        for image_path in self.image_paths:
            img = cv2.imread(image_path)
            hist = self.calculate_histogram(img)
            self.features.append(hist.flatten())

    def calculate_histogram(self, image):
        hist = cv2.calcHist(
            [image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]
        )
        cv2.normalize(hist, hist)
        return hist

    def search_similar_images(self, query_image_path, top_k=10):
        query_img = cv2.imread(query_image_path)
        query_hist = self.calculate_histogram(query_img).flatten()

        distances = euclidean_distances([query_hist], self.features).flatten()
        sorted_indices = np.argsort(distances)[:top_k]

        return [
            (self.image_paths[i], os.path.basename(self.image_paths[i]))
            for i in sorted_indices
        ]

    def plot_results(self, query_image_path, result_image_info):
        fig, axes = plt.subplots(2, 5, figsize=(15, 6))

        # Plot query image
        query_img = cv2.imread(query_image_path)
        axes[0, 0].imshow(cv2.cvtColor(query_img, cv2.COLOR_BGR2RGB))
        axes[0, 0].set_title("Query Image")
        axes[0, 0].axis("off")

        # Plot similar images
        for i, (result_image_path, result_image_name) in enumerate(result_image_info):
            img = cv2.imread(result_image_path)
            axes[i // 5, i % 5].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            axes[i // 5, i % 5].set_title(result_image_name)
            axes[i // 5, i % 5].axis("off")

        plt.tight_layout()
        plt.show()

### Usage Example

- **Dataset and Query Image:**

Load images from a dataset path (`/kaggle/input/corel-images/dataset/training_set`). Then, specify a query image path such as (`/kaggle/input/corel-images/dataset/horses_test/700.jpg`).

- **Image Loading and Feature Extraction:**

Load and display the query image.
Load all images from the dataset and extract their color histograms.

- **Image Retrieval:**

Search for the top 10 most similar images to the query image based on histogram comparison.

- **Result Visualization:**

Display the query image and the top 10 retrieved results in a grid arrangement.


In [ ]:
dataset_path = "/kaggle/input/corel-images/dataset/training_set"
cbir_system = CBIRSystem(dataset_path)
cbir_system.load_images()
cbir_system.extract_features()
print("Number of images in the dataset: ", len(cbir_system.image_paths))

In [ ]:
query_image_path = "/kaggle/input/corel-images/dataset/horses_test/700.jpg"
query_img = cv2.imread(query_image_path)
plt.imshow(cv2.cvtColor(query_img, cv2.COLOR_BGR2RGB))
plt.title("Query Image")
plt.axis("off")
plt.show()
result_images = cbir_system.search_similar_images(query_image_path, top_k=10)
cbir_system.plot_results(query_image_path, result_images)

### Key Points

The code uses color histograms for image features, providing a basic CBIR method. Euclidean distance is used for similarity comparison, but may not accurately capture perceptual differences. The code is modular and well-organized, and can be adapted to use different features, similarity metrics, and datasets. Potential extensions include feature exploration and exploring alternative similarity metrics.


> ### **Task 2:**
>
> Implement the CBIR system using Color Histogram as an image representation. Experiment with 120 bins, 180 bins and with 360 bins. Use Euclidean as distant measure and compute precision, recall, F1 score, and time for each experiment. Construct a Receiver Operating Characteristic (ROC) curve by varying the retrieval threshold. Calculate the Area Under the Curve (AUC) to measure the overall performance across different threshold settings. Note that you need to compute these measures as an average of at least 10 different quires.


***NOTE***: As it was told to us, the main objective of this task is to determine the effectiveness of the number of bins. So, any three different values, up to 256 bins, can be used. For simplification, I used 8 bins, 100 bins, and 150 bins.

The code includes a utility function for managing feature files, measuring memory usage, and execution time. The `delete_features_file` function deletes a specified feature file, while the measure_memory_usage_and_time_spent function measures memory usage and time spent during another function's execution. These utilities are useful for optimizing data processing pipelines, especially in large datasets or resource-intensive operations. The `psutil` library allows monitoring of memory usage, while the `time` module measures execution time. The code emphasizes resource management and optimization in image processing and retrieval tasks.


In [ ]:
import os
import psutil
import time

def delete_features_file(features_file="features.pkl"):
    try:
        os.remove(features_file)
    except FileNotFoundError:
        print(f"{features_file} not found.")

dataset_path = "/kaggle/input/corel-images/dataset/training_set"
query_images_path = "/kaggle/input/corel-images/dataset/horses_test"

def measure_memory_usage_and_time_spent(func, *args, **kwargs):
    start_time = time.time()
    before_memory = psutil.virtual_memory().used / (1024 ** 2)
    result = func(*args, **kwargs)
    after_memory = psutil.virtual_memory().used / (1024 ** 2)
    memory_usage = after_memory - before_memory
    print(f"Memory usage during the operation: {memory_usage:.2f} MB")
    end_time = time.time()
    elapsed_time_seconds = end_time - start_time
    hours, remainder = divmod(elapsed_time_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    time_format = "{:02}:{:02}:{:02}".format(int(hours), int(minutes), int(seconds))
    print("Time spent: {}".format(time_format))
    return result

This code consumes a lot of memory, so I modified the basic `CBIRSystem class` to save the features extracted on a file called `features.pkl` to avoid crashes. It will load images from the dataset, extract features using a color histogram with a specific number of bins each time, then search for similar images as it will sort the images according to Euclidean distance, take the top 10 images, and plot them. The `evaluate_performance` function evaluates system performance using precision, recall, F1-score, and ROC-AUC metrics. It iterates through query images, calculates distances, computes precision, recall, and F1-score for each query, aggregates scores, creates a ROC curve, calculates AUC, and displays performance metrics. This comprehensive assessment of the CBIR system's retrieval performance is essential for analysis.


In [ ]:
import os
import cv2
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
from sklearn.metrics import auc
import pickle

class CBIRSystem:
    def __init__(self, dataset_path, features_file="features.pkl"):
        self.dataset_path = dataset_path
        self.image_paths = []
        self.features_file = features_file
        self.features = None

    def load_images(self):
        self.image_paths = [
            os.path.join(root, file)
            for root, dirs, files in os.walk(self.dataset_path)
            for file in files
            if file.lower().endswith((".png", ".jpg", ".jpeg"))
        ]

    def extract_features(self, bin_size=8):
        with open(self.features_file, 'wb') as f:
            for image_path in self.image_paths:
                img = cv2.imread(image_path)
                r_img = cv2.resize(img, (5, 5))
                hist = self.calculate_histogram(r_img, bin_size)
                pickle.dump(hist.flatten(), f)

    def load_features(self):
        with open(self.features_file, 'rb') as f:
            self.features = [pickle.load(f) for _ in range(len(self.image_paths))]

    def clear_features(self):
        self.features = None

    def calculate_histogram(self, image, bin_size):
        hist = cv2.calcHist(
            [image], [0, 1, 2], None, [bin_size, bin_size, bin_size], [0, 256, 0, 256, 0, 256]
        )
        cv2.normalize(hist, hist)
        return hist

    def search_similar_images(self, query_image_path, top_k=10, bin_size=8):
        query_hist = self.calculate_histogram(cv2.imread(query_image_path), bin_size).flatten()
        distances = euclidean_distances([query_hist], self.features).flatten()
        sorted_indices = np.argsort(distances)[:top_k]
        return [(self.image_paths[i], os.path.basename(self.image_paths[i])) for i in sorted_indices]

    def plot_results(self, query_image_path, result_image_info):
        fig, axes = plt.subplots(2, 5, figsize=(15, 6))
        query_img = cv2.imread(query_image_path)

        axes[0, 0].imshow(cv2.cvtColor(query_img, cv2.COLOR_BGR2RGB))
        axes[0, 0].set_title("Query Image")
        axes[0, 0].axis("off")

        for i, (result_image_path, result_image_name) in enumerate(result_image_info):
            img = cv2.imread(result_image_path)
            axes[i // 5, i % 5].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            axes[i // 5, i % 5].set_title(result_image_name)
            axes[i // 5, i % 5].axis("off")

        plt.tight_layout()
        plt.show()

    def calculate_euclidean_distances(self, query_image_path, bin_size=8):
        query_hist = self.calculate_histogram(cv2.imread(query_image_path), bin_size).flatten()
        distances = euclidean_distances([query_hist], self.features).flatten()
        return distances

    def evaluate_performance(self, query_images_path, bin_size=8):
        all_true_labels = []
        all_distances = []
        precision_list = []
        recall_list = []
        f1_score_list = []
        ground_truth_labels = [1 if 'horses' in path and '710.jpg' <= path.split(os.path.sep)[-1] <= '799.jpg' else 0 for path in self.image_paths]

        TP_list, FP_list, TN_list, FN_list = [], [], [], []

        for query_image in os.listdir(query_images_path):
            TP, FP, TN, FN = 0, 0, 0, 0
            query_image_path = os.path.join(query_images_path, query_image)
            distances = self.calculate_euclidean_distances(query_image_path, bin_size=bin_size)
            sorted_indices = np.argsort(distances)
            true_labels = ground_truth_labels

            for i in range(10):
                if true_labels[sorted_indices[i]] == 1:
                    TP += 1
                else:
                    FP += 1

            for i in range(10, len(self.image_paths)):
                if true_labels[sorted_indices[i]] == 0:
                    TN += 1
                else:
                    FN += 1

            TP_list.append(TP)
            FP_list.append(FP)
            TN_list.append(TN)
            FN_list.append(FN)

            all_true_labels.extend(true_labels)
            all_distances.extend(distances)

            precision = TP / (TP + FP) if TP + FP != 0 else 0
            recall = TP / (TP + FN) if TP + FN != 0 else 0
            f1_score = 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0

            # Store metrics for averaging
            precision_list.append(precision)
            recall_list.append(recall)
            f1_score_list.append(f1_score)

            print("Metrics for Query:", query_image)
            print("Precision:", precision)
            print("Recall:", recall)
            print("F1 Score:", f1_score)

        # Calculate and print average Precision, Recall, and F1 Score
        avg_precision = sum(precision_list) / len(precision_list)
        avg_recall = sum(recall_list) / len(recall_list)
        avg_f1_score = sum(f1_score_list) / len(f1_score_list)

        print("Average Precision:", avg_precision)
        print("Average Recall:", avg_recall)
        print("Average F1 Score:", avg_f1_score)
        

        sorted_indices_all = np.argsort(all_distances)
        total_positives, total_negatives = sum(all_true_labels), len(all_true_labels) - sum(all_true_labels)

        fpr = [0]
        tpr = [0]
        for i in range(len(all_true_labels)):
            if all_true_labels[sorted_indices_all[i]] == 1:
                tpr.append(tpr[-1] + 1 / total_positives)
                fpr.append(fpr[-1])
            else:
                tpr.append(tpr[-1])
                fpr.append(fpr[-1] + 1 / total_negatives)

        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve with AUC = {:.2f}'.format(auc(fpr, tpr)))
        plt.legend(loc="lower right")
        plt.show()

In [ ]:
def cbir_system_using_color_histogram(dataset_path, query_images_path, bin_size, features_file="features.pkl"):
    cbir_system = CBIRSystem(dataset_path, features_file)
    cbir_system.load_images()
    cbir_system.extract_features(bin_size=bin_size)
    cbir_system.load_features()  # Load features before using search_similar_images

    print("Number of images in the dataset: ", len(cbir_system.image_paths))

    for query_image in os.listdir(query_images_path):
        query_image_path = os.path.join(query_images_path, query_image)

        plt.imshow(cv2.cvtColor(cv2.imread(query_image_path), cv2.COLOR_BGR2RGB))
        plt.title("Query Image")
        plt.axis("off")
        plt.show()

        result_images = cbir_system.search_similar_images(query_image_path, top_k=10, bin_size=bin_size)
        cbir_system.plot_results(query_image_path, result_images)

    cbir_system.evaluate_performance(query_images_path, bin_size=bin_size)
    return cbir_system

Kaggle has only 19.5 GB of storage, so I called the function `delete_features_file` to clear the space for each new run.

Then, a CBIR system using a color histogram was used with three different values of bins to compare them.


In [ ]:
delete_features_file()
measure_memory_usage_and_time_spent(cbir_system_using_color_histogram, dataset_path, query_images_path, bin_size=8)

In [ ]:
delete_features_file()
measure_memory_usage_and_time_spent(cbir_system_using_color_histogram, dataset_path, query_images_path, bin_size=100)

In [ ]:
delete_features_file()
measure_memory_usage_and_time_spent(cbir_system_using_color_histogram, dataset_path, query_images_path, bin_size=150)

## Summary of Results

**8 Bins:**

- Average precision: 0.83
- Average Recall: 0.092
- Average F1 Score: 0.166
- Memory Usage: 37.60 MB
- Time Spent: 00:00:27
- AUC: 0.78

**100 Bins:**

- Average precision: 0.38
- Average Recall: 0.042
- Average F1 Score: 0.076
- Memory Usage: 3812.55 MB
- Time Spent: 00:02:46
- AUC: 0.67

**150 Bins:**

- Average precision: 0.33
- Average Recall: 0.037
- Average F1 Score: 0.066
- Memory Usage: 12808.48 MB
- Time Spent: 00:08:25
- AUC: 0.65

## Comparison and Analysis

**Effect of Bin Size:**

The CBIR system with 8 bins shows higher precision, recall, and F1 score, suggesting that increasing the number of bins does not significantly improve performance in this scenario.
However, with 100 bins and 150 bins, the performance metrics dropped, indicating that a higher number of bins may result in a loss of discriminative information in the color histograms.

**Memory Usage and Execution Time:**

As the number of bins increases, both memory usage and execution time also increase significantly. This is expected because a higher number of bins leads to larger histograms and increased computational requirements during feature extraction.

**AUC (Area Under the ROC Curve):**

AUC values provide a measure of the system's ability to distinguish between positive and negative cases. In this case, a lower AUC with more bins suggests that the increased granularity in the color histograms might be introducing noise, making it harder to discriminate between relevant and irrelevant images.
In general, color histograms provide limited information about color distribution, overlooking spatial relationships, textures, and object shapes. They can be affected by lighting conditions and color variations, potentially affecting retrieval accuracy. Additionally, they may not accurately represent images with diverse color palettes and complex visual structures.

**Why Fewer Bins Might Be Better?**

- Robustness to Noise: Fewer bins provide more robust features and are less sensitive to minor color variations due to lighting or image compression.
- Computational Efficiency: Smaller histograms require less memory and processing time, leading to faster retrieval and lower resource consumption.
- Generalization: Coarser color representations can capture broader similarities, potentially enhancing the retrieval of images with slight color differences.


> ### **Task 3:** in this task you need to experiment with color moments as following:
>
> **3.1:** Implement the CBIR system using Color Moments (mean, standard deviation, and skewness) as an
> image representation. Use Euclidean as distant measure and assign equal weights to each moment.
> Compute precision, recall, F1 score, and time. Calculate the Area Under the Curve (AUC) to measure the
> overall performance across different threshold settings. Note that you need to compute these measures
> as an average of at least 10 different quires.


The `calculate_color_moments` method efficiently captures the color characteristics of an image by computing its mean, standard deviation, and skewness values. Utilizing the `np.mean` function, it calculates the average color values for each channel (red, green, and blue), providing insight into the central tendency of the color distribution. The computation of the standard deviation with `np.std` offers information about the spread or variation of color values around the mean, highlighting color diversity within the image. The method further reshapes the image into a 2D array and employs the `skew` function, likely from SciPy, to determine the skewness of each color channel. Skewness serves as a measure of asymmetry in the color distribution, indicating any bias towards higher or lower color values. Ultimately, the calculated mean, standard deviation, and skewness values are concatenated into a single feature vector using `np.concatenate`. This concise vector encapsulates the color moments of the image, providing a statistical summary of its color distribution. Such a representation is particularly valuable for image retrieval and various image processing tasks, offering a nuanced understanding of color characteristics beyond what is achievable with traditional color histograms.


In [ ]:
import os
import cv2
import numpy as np
from scipy.stats import skew
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, auc
import time
import psutil

class CBIRSystem:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.image_paths = []
        self.features = []

    def load_images(self):
        for root, dirs, files in os.walk(self.dataset_path):
            for file in files:
                if file.lower().endswith((".png", ".jpg", ".jpeg")):
                    image_path = os.path.join(root, file)
                    self.image_paths.append(image_path)

    def extract_features(self):
        for image_path in self.image_paths:
            img = cv2.imread(image_path)
            moments = self.calculate_color_moments(img)
            self.features.append(moments)

    def calculate_color_moments(self, image):
        mean = np.mean(image, axis=(0, 1))
        std_dev = np.std(image, axis=(0, 1))
        skewness = skew(image.reshape(-1, 3), axis=0)
        return np.concatenate((mean, std_dev, skewness))

    def search_similar_images(self, query_image_path, top_k=10):
        query_img = cv2.imread(query_image_path)
        query_moments = self.calculate_color_moments(query_img)

        distances = euclidean_distances([query_moments], self.features).flatten()
        sorted_indices = np.argsort(distances)[:top_k]

        return [
            (self.image_paths[i], os.path.basename(self.image_paths[i]))
            for i in sorted_indices
        ]

    def plot_results(self, query_image_path, result_image_info):
        fig, axes = plt.subplots(2, 5, figsize=(15, 6))

        # Plot query image
        query_img = cv2.imread(query_image_path)
        axes[0, 0].imshow(cv2.cvtColor(query_img, cv2.COLOR_BGR2RGB))
        axes[0, 0].set_title("Query Image")
        axes[0, 0].axis("off")

        # Plot similar images
        for i, (result_image_path, result_image_name) in enumerate(result_image_info):
            img = cv2.imread(result_image_path)
            axes[i // 5, i % 5].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            axes[i // 5, i % 5].set_title(result_image_name)
            axes[i // 5, i % 5].axis("off")

        plt.tight_layout()
        plt.show()

    def calculate_euclidean_distances(self, query_image_path):
        query_img = cv2.imread(query_image_path)
        query_moments = self.calculate_color_moments(query_img)

        distances = euclidean_distances([query_moments], self.features).flatten()
        return distances

    def evaluate_performance(self, query_images_path):
        all_true_labels = []
        all_distances = []
        precision_list = []
        recall_list = []
        f1_score_list = []
        
        # Load ground truth
        ground_truth_labels = [1 if 'horses' in path and '710.jpg' <= path.split(os.path.sep)[-1] <= '799.jpg' else 0 for path in self.image_paths]

        # Initialize arrays to store counts for each query
        TP_list = []
        FP_list = []
        TN_list = []
        FN_list = []

        # Loop through each query image
        for query_image in os.listdir(query_images_path):
            # Initialize counts for the current query
            TP = 0
            FP = 0
            TN = 0
            FN = 0

            query_image_path = os.path.join(query_images_path, query_image)
            distances = self.calculate_euclidean_distances(query_image_path)
            sorted_indices = np.argsort(distances)

            true_labels = ground_truth_labels  # Use ground truth labels for evaluation

            # Count the relevant images in the top-k results
            for i in range(10):
                if true_labels[sorted_indices[i]] == 1:
                    TP += 1
                else:
                    FP += 1

            # Count the non-relevant images after the top-k results
            for i in range(10, len(self.image_paths)):
                if true_labels[sorted_indices[i]] == 0:
                    TN += 1
                else:
                    FN += 1

            # Append counts for the current query to the arrays
            TP_list.append(TP)
            FP_list.append(FP)
            TN_list.append(TN)
            FN_list.append(FN)

            # Store results for averaging
            all_true_labels.extend(true_labels)
            all_distances.extend(distances)

            # Calculate metrics for the current query
            precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, distances <= distances[sorted_indices[9]], average='binary')

            # Store metrics for averaging
            precision_list.append(precision)
            recall_list.append(recall)
            f1_score_list.append(f1_score)

            print("Metrics for Query:", query_image)
            print("Precision:", precision)
            print("Recall:", recall)
            print("F1 Score:", f1_score)

        # Calculate and print average Precision, Recall, and F1 Score
        avg_precision = sum(precision_list) / len(precision_list)
        avg_recall = sum(recall_list) / len(recall_list)
        avg_f1_score = sum(f1_score_list) / len(f1_score_list)

        print("Average Precision:", avg_precision)
        print("Average Recall:", avg_recall)
        print("Average F1 Score:", avg_f1_score)
        

        # Manually calculate FPR and TPR
        sorted_indices_all = np.argsort(all_distances)
        total_positives = sum(all_true_labels)
        total_negatives = len(all_true_labels) - total_positives

        fpr = [0]
        tpr = [0]
        for i in range(len(all_true_labels)):
            if all_true_labels[sorted_indices_all[i]] == 1:
                tpr.append(tpr[-1] + 1 / total_positives)
                fpr.append(fpr[-1])
            else:
                tpr.append(tpr[-1])
                fpr.append(fpr[-1] + 1 / total_negatives)

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve with AUC = {:.2f}'.format(auc(fpr, tpr)))
        plt.legend(loc="lower right")
        plt.show()

In [ ]:
def cbir_system_using_color_moments(dataset_path, query_images_path):
    cbir_system = CBIRSystem(dataset_path)
    cbir_system.load_images()
    cbir_system.extract_features()
    print("Number of images in the dataset: ", len(cbir_system.image_paths))

    for query_image in os.listdir(query_images_path):
        query_image_path = os.path.join(query_images_path, query_image)

        plt.imshow(cv2.cvtColor(cv2.imread(query_image_path), cv2.COLOR_BGR2RGB))
        plt.title("Query Image")
        plt.axis("off")
        plt.show()

        result_images = cbir_system.search_similar_images(query_image_path, top_k=10)
        cbir_system.plot_results(query_image_path, result_images)

    # Evaluate performance
    cbir_system.evaluate_performance(query_images_path)
    return cbir_system

In [ ]:
measure_memory_usage_and_time_spent(cbir_system_using_color_moments, dataset_path, query_images_path)

> **3.2:** Same as task **3.1** but with different weights. You need to give a weigh relative to the important of the
> moment.


This task is like task 3.1 but with additional attribut `weights`.


In [ ]:
import os
import cv2
import numpy as np
from scipy.stats import skew
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, auc

class CBIRSystem:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.image_paths = []
        self.features = []

    def load_images(self):
        for root, dirs, files in os.walk(self.dataset_path):
            for file in files:
                if file.lower().endswith((".png", ".jpg", ".jpeg")):
                    image_path = os.path.join(root, file)
                    self.image_paths.append(image_path)

    def extract_features(self, weights):
        for image_path in self.image_paths:
            img = cv2.imread(image_path)
            moments = self.calculate_color_moments(img, weights)
            self.features.append(moments)

    def calculate_color_moments(self, image, weights):
        mean = np.mean(image, axis=(0, 1))
        std_dev = np.std(image, axis=(0, 1))
        skewness = skew(image.reshape(-1, 3), axis=0)
        
        weighted_moments = weights[0] * mean + weights[1] * std_dev + weights[2] * skewness
        return weighted_moments

    def search_similar_images(self, query_image_path, weights, top_k=10):
        query_img = cv2.imread(query_image_path)
        query_moments = self.calculate_color_moments(query_img, weights)

        distances = euclidean_distances([query_moments], self.features).flatten()
        sorted_indices = np.argsort(distances)[:top_k]

        return [
            (self.image_paths[i], os.path.basename(self.image_paths[i]))
            for i in sorted_indices
        ]

    def plot_results(self, query_image_path, result_image_info):
        fig, axes = plt.subplots(2, 5, figsize=(15, 6))

        # Plot query image
        query_img = cv2.imread(query_image_path)
        axes[0, 0].imshow(cv2.cvtColor(query_img, cv2.COLOR_BGR2RGB))
        axes[0, 0].set_title("Query Image")
        axes[0, 0].axis("off")

        # Plot similar images
        for i, (result_image_path, result_image_name) in enumerate(result_image_info):
            img = cv2.imread(result_image_path)
            axes[i // 5, i % 5].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            axes[i // 5, i % 5].set_title(result_image_name)
            axes[i // 5, i % 5].axis("off")

        plt.tight_layout()
        plt.show()

    def calculate_euclidean_distances(self, query_image_path, weights):
        query_img = cv2.imread(query_image_path)
        query_moments = self.calculate_color_moments(query_img, weights)

        distances = euclidean_distances([query_moments], self.features).flatten()
        return distances

    def evaluate_performance(self, query_images_path, weights):
        all_true_labels = []
        all_distances = []
        precision_list = []
        recall_list = []
        f1_score_list = []
        
        # Load ground truth
        ground_truth_labels = [1 if 'horses' in path and '710.jpg' <= path.split(os.path.sep)[-1] <= '799.jpg' else 0 for path in self.image_paths]

        # Initialize arrays to store counts for each query
        TP_list = []
        FP_list = []
        TN_list = []
        FN_list = []

        # Loop through each query image
        for query_image in os.listdir(query_images_path):
            # Initialize counts for the current query
            TP = 0
            FP = 0
            TN = 0
            FN = 0

            query_image_path = os.path.join(query_images_path, query_image)
            distances = self.calculate_euclidean_distances(query_image_path, weights)
            sorted_indices = np.argsort(distances)

            true_labels = ground_truth_labels  # Use ground truth labels for evaluation

            # Count the relevant images in the top-k results
            for i in range(10):
                if true_labels[sorted_indices[i]] == 1:
                    TP += 1
                else:
                    FP += 1

            # Count the non-relevant images after the top-k results
            for i in range(10, len(self.image_paths)):
                if true_labels[sorted_indices[i]] == 0:
                    TN += 1
                else:
                    FN += 1

            # Append counts for the current query to the arrays
            TP_list.append(TP)
            FP_list.append(FP)
            TN_list.append(TN)
            FN_list.append(FN)

            # Store results for averaging
            all_true_labels.extend(true_labels)
            all_distances.extend(distances)

            # Calculate metrics for the current query
            precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, distances <= distances[sorted_indices[9]], average='binary')

            # Store metrics for averaging
            precision_list.append(precision)
            recall_list.append(recall)
            f1_score_list.append(f1_score)

            print("Metrics for Query:", query_image)
            print("Precision:", precision)
            print("Recall:", recall)
            print("F1 Score:", f1_score)

        # Calculate and print average Precision, Recall, and F1 Score
        avg_precision = sum(precision_list) / len(precision_list)
        avg_recall = sum(recall_list) / len(recall_list)
        avg_f1_score = sum(f1_score_list) / len(f1_score_list)

        print("Average Precision:", avg_precision)
        print("Average Recall:", avg_recall)
        print("Average F1 Score:", avg_f1_score)
        

        # Manually calculate FPR and TPR
        sorted_indices_all = np.argsort(all_distances)
        total_positives = sum(all_true_labels)
        total_negatives = len(all_true_labels) - total_positives

        fpr = [0]
        tpr = [0]
        for i in range(len(all_true_labels)):
            if all_true_labels[sorted_indices_all[i]] == 1:
                tpr.append(tpr[-1] + 1 / total_positives)
                fpr.append(fpr[-1])
            else:
                tpr.append(tpr[-1])
                fpr.append(fpr[-1] + 1 / total_negatives)

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve with AUC = {:.2f}'.format(auc(fpr, tpr)))
        plt.legend(loc="lower right")
        plt.show()

**Choosing weights:**

In this case, we have three moments: mean, standard deviation, and skewness. The weights can be assigned based on the relative importance I want to give to each moment in this scenario.

`w_mean`: 0.5 (high weight, indicating that the mean is a crucial measure of central tendency).

`w_std_dev`: 0.3 (medium weight, representing the importance of color variation around the mean).

`w_skewness`: 0.2 (lower weight, as skewness provides additional information but may be less critical than mean and standard deviation).


In [ ]:
def cbir_system_using_color_moments(dataset_path, query_images_path):
    # weights for mean, std_dev, and skewness
    weights = [0.5, 0.3, 0.2]

    cbir_system = CBIRSystem(dataset_path)
    cbir_system.load_images()
    cbir_system.extract_features(weights)
    print("Number of images in the dataset: ", len(cbir_system.image_paths))

    for query_image in os.listdir(query_images_path):
        query_image_path = os.path.join(query_images_path, query_image)

        plt.imshow(cv2.cvtColor(cv2.imread(query_image_path), cv2.COLOR_BGR2RGB))
        plt.title("Query Image")
        plt.axis("off")
        plt.show()

        result_images = cbir_system.search_similar_images(query_image_path, weights, top_k=10)
        cbir_system.plot_results(query_image_path, result_images)

    # Evaluate performance
    cbir_system.evaluate_performance(query_images_path, weights)
    return cbir_system


In [ ]:
measure_memory_usage_and_time_spent(cbir_system_using_color_moments, dataset_path, query_images_path)

> **3.3:** Same as task **3.2** but with the addition of more Moments including Median, Mode, and Kurtosis.


In [ ]:
import os
import cv2
import numpy as np
from scipy.stats import skew, kurtosis
from scipy import stats
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, auc

class CBIRSystem:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.image_paths = []
        self.features = []

    def load_images(self):
        for root, dirs, files in os.walk(self.dataset_path):
            for file in files:
                if file.lower().endswith((".png", ".jpg", ".jpeg")):
                    image_path = os.path.join(root, file)
                    self.image_paths.append(image_path)

    def extract_features(self, weights):
        for image_path in self.image_paths:
            img = cv2.imread(image_path)
            moments = self.calculate_color_moments(img, weights)
            self.features.append(moments)

    def calculate_color_moments(self, image, weights):
        mean = np.mean(image, axis=(0, 1))
        std_dev = np.std(image, axis=(0, 1))
        skewness = skew(image.reshape(-1, 3), axis=0)

        # Calculate median, mode, and kurtosis
        median = np.median(image, axis=(0, 1))
        mode = stats.mode(image, axis=(0, 1)).mode  # Extract mode values
        kurtosis = stats.kurtosis(image.reshape(-1, 3), axis=0)

        # Combine all moments with weights
        weighted_moments = weights[0] * mean + weights[1] * std_dev + weights[2] * skewness + \
                             weights[3] * median + weights[4] * mode + weights[5] * kurtosis
        return weighted_moments

    def search_similar_images(self, query_image_path, weights, top_k=10):
        query_img = cv2.imread(query_image_path)
        query_moments = self.calculate_color_moments(query_img, weights)

        distances = euclidean_distances([query_moments], self.features).flatten()
        sorted_indices = np.argsort(distances)[:top_k]

        return [
            (self.image_paths[i], os.path.basename(self.image_paths[i]))
            for i in sorted_indices
        ]

    def plot_results(self, query_image_path, result_image_info):
        fig, axes = plt.subplots(2, 5, figsize=(15, 6))

        # Plot query image
        query_img = cv2.imread(query_image_path)
        axes[0, 0].imshow(cv2.cvtColor(query_img, cv2.COLOR_BGR2RGB))
        axes[0, 0].set_title("Query Image")
        axes[0, 0].axis("off")

        # Plot similar images
        for i, (result_image_path, result_image_name) in enumerate(result_image_info):
            img = cv2.imread(result_image_path)
            axes[i // 5, i % 5].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            axes[i // 5, i % 5].set_title(result_image_name)
            axes[i // 5, i % 5].axis("off")

        plt.tight_layout()
        plt.show()

    def calculate_euclidean_distances(self, query_image_path, weights):
        query_img = cv2.imread(query_image_path)
        query_moments = self.calculate_color_moments(query_img, weights)

        distances = euclidean_distances([query_moments], self.features).flatten()
        return distances

    def evaluate_performance(self, query_images_path, weights):
        all_true_labels = []
        all_distances = []
        precision_list = []
        recall_list = []
        f1_score_list = []
        
        # Load ground truth
        ground_truth_labels = [1 if 'horses' in path and '710.jpg' <= path.split(os.path.sep)[-1] <= '799.jpg' else 0 for path in self.image_paths]

        # Initialize arrays to store counts for each query
        TP_list = []
        FP_list = []
        TN_list = []
        FN_list = []

        # Loop through each query image
        for query_image in os.listdir(query_images_path):
            # Initialize counts for the current query
            TP = 0
            FP = 0
            TN = 0
            FN = 0

            query_image_path = os.path.join(query_images_path, query_image)
            distances = self.calculate_euclidean_distances(query_image_path, weights)
            sorted_indices = np.argsort(distances)

            true_labels = ground_truth_labels  # Use ground truth labels for evaluation

            # Count the relevant images in the top-k results
            for i in range(10):
                if true_labels[sorted_indices[i]] == 1:
                    TP += 1
                else:
                    FP += 1

            # Count the non-relevant images after the top-k results
            for i in range(10, len(self.image_paths)):
                if true_labels[sorted_indices[i]] == 0:
                    TN += 1
                else:
                    FN += 1

            # Append counts for the current query to the arrays
            TP_list.append(TP)
            FP_list.append(FP)
            TN_list.append(TN)
            FN_list.append(FN)

            # Store results for averaging
            all_true_labels.extend(true_labels)
            all_distances.extend(distances)

            # Calculate metrics for the current query
            precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, distances <= distances[sorted_indices[9]], average='binary')

            # Store metrics for averaging
            precision_list.append(precision)
            recall_list.append(recall)
            f1_score_list.append(f1_score)

            print("Metrics for Query:", query_image)
            print("Precision:", precision)
            print("Recall:", recall)
            print("F1 Score:", f1_score)

        # Calculate and print average Precision, Recall, and F1 Score
        avg_precision = sum(precision_list) / len(precision_list)
        avg_recall = sum(recall_list) / len(recall_list)
        avg_f1_score = sum(f1_score_list) / len(f1_score_list)

        print("Average Precision:", avg_precision)
        print("Average Recall:", avg_recall)
        print("Average F1 Score:", avg_f1_score)
        

        # Manually calculate FPR and TPR
        sorted_indices_all = np.argsort(all_distances)
        total_positives = sum(all_true_labels)
        total_negatives = len(all_true_labels) - total_positives

        fpr = [0]
        tpr = [0]
        for i in range(len(all_true_labels)):
            if all_true_labels[sorted_indices_all[i]] == 1:
                tpr.append(tpr[-1] + 1 / total_positives)
                fpr.append(fpr[-1])
            else:
                tpr.append(tpr[-1])
                fpr.append(fpr[-1] + 1 / total_negatives)

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve with AUC = {:.2f}'.format(auc(fpr, tpr)))
        plt.legend(loc="lower right")
        plt.show()

**Choosing weights:**

`Mean and Standard Deviation`: Given relatively higher weights of 0.25 and 0.2, respectively, mean and standard deviation continue to play significant roles in capturing the central tendency and variation in color distribution.

`Skewness`: While skewness retains importance, its weight has been adjusted to 0.15, reflecting its role in characterizing the asymmetry of the color distribution.

`Median`: The inclusion of the median with a weight of 0.15 indicates the importance of capturing central tendency, especially in the presence of outliers.

`Mode and Kurtosis`: These features are assigned weights of 0.1 and 0.15, respectively, suggesting their roles in representing specific characteristics of the color distribution.


In [ ]:
def cbir_system_using_color_moments(dataset_path, query_images_path):
    # weights for mean, std_dev, skewness, median, mode, and kurtosis
    weights = [0.25, 0.2, 0.15, 0.15, 0.1, 0.15]

    cbir_system = CBIRSystem(dataset_path)
    cbir_system.load_images()
    cbir_system.extract_features(weights)
    print("Number of images in the dataset: ", len(cbir_system.image_paths))

    for query_image in os.listdir(query_images_path):
        query_image_path = os.path.join(query_images_path, query_image)

        plt.imshow(cv2.cvtColor(cv2.imread(query_image_path), cv2.COLOR_BGR2RGB))
        plt.title("Query Image")
        plt.axis("off")
        plt.show()

        result_images = cbir_system.search_similar_images(query_image_path, weights, top_k=10)
        cbir_system.plot_results(query_image_path, result_images)

    # Evaluate performance
    cbir_system.evaluate_performance(query_images_path, weights)
    return cbir_system


In [ ]:
measure_memory_usage_and_time_spent(cbir_system_using_color_moments, dataset_path, query_images_path)

## Summary of Results

**Case 1 with Mean, Std_Dev, and Skewness**:

- Average precision: 0.71
- Average Recall: 0.0789
- Average F1 Score: 0.142
- Memory Usage: 49.57 MB
- Time Spent: 00:00:58
- AUC: 0.77

**Case 2 with Std_Dev, Skewness, Median, Mode, and Kurtosis**:

- Average precision: 0.67
- Average Recall: 0.0744
- Average F1 Score: 0.134
- Memory Usage: 30.48 MB
- Time Spent: 00:01:36
- AUC: 0.78

## Analysis

**Case 1**:

The higher weight assigned to the mean (0.5) suggests a strong emphasis on the central tendency of the color distribution.
The overall performance metrics, including average precision, recall, and F1 score, indicate reasonable retrieval results.

**Case 2**:

The weights are more evenly distributed across different color moments.
The inclusion of additional moments (median, mode, and kurtosis) allows for a more comprehensive characterization of the color distribution.
Despite a slightly lower average precision, the system achieves a comparable AUC, indicating a good balance between precision and recall.

## Conclusion

Both cases gave good results with low memory usage, but with a lower time for case 1 and a slightly higher AUC for case 2.

CBIR with color moments outperforms CBIR with color histograms due to their distinct characteristics and information capture. Histograms provide a global representation of color distribution but lack spatial information and discriminative power. Color moments provide a more detailed representation of color patterns, are more sensitive to spatial arrangement, provide higher-level information, and are flexible. The choice between these methods depends on the image's nature, retrieval system goals, and dataset characteristics.


> ### **Task 4:**
>
> Try to improve the performance of the CBIR system using other image representation techniques.
> 

In this part, I tested several algorithms and the HOG algorithm gave me the best results. The algorithm calculates Histogram of Oriented Gradients (`HOG`) features for an image, which is a feature descriptor commonly used in computer vision and image processing for object detection and recognition. The algorithm involves input, calculation, and return of HOG features. The key parameters of the `hog` function include orientations, cell size, block size, and visualization.

The HOG features can be used as a descriptor for the image, representing the spatial distribution of gradient orientations. They are particularly useful for representing the structure and texture of objects in an image. The algorithm's parameters, such as orientations, cell size, and block size, can be adjusted based on the specific requirements of the application.

The HOG algorithm's implementation includes gradient computation, cellular grid, histogram creation, block normalization, and feature vector. The algorithm calculates the horizontal and vertical gradients for each pixel in the image using a derivative filter, obtaining the gradient magnitude and orientation for each pixel. It then divides the image into small, overlapping cells, creates a histogram of gradient orientations for each cell, and normalizes the histograms within each block to reduce the effect of lighting variations and enhance contrast.

The code implementation calls the `feature.hog` function from a library `presumably scikit-image` to compute HOG features. The parameters include orientations, cell size, block size, visualization, and the return of the HOG feature vector.

HOG features are robust to lighting variations and small changes in object position, capturing shape information effectively, making them useful for object detection and recognition tasks. However, parameter choices can impact performance and should be tuned for specific applications.


In [ ]:
import os
import cv2
import numpy as np
from skimage import feature
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, auc

class CBIRSystemHOG:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.image_paths = []
        self.features = []

    def load_images(self):
        for root, dirs, files in os.walk(self.dataset_path):
            for file in files:
                if file.lower().endswith((".png", ".jpg", ".jpeg")):
                    image_path = os.path.join(root, file)
                    self.image_paths.append(image_path)

    def extract_features(self):
        for image_path in self.image_paths:
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            features = self.calculate_hog_features(img)
            self.features.append(features)

    def calculate_hog_features(self, image):
        # Calculate HOG features
        hog_features, _ = feature.hog(image, orientations=8, pixels_per_cell=(16, 16),
                                      cells_per_block=(1, 1), block_norm="L2-Hys", visualize=True)

        return hog_features

    def search_similar_images(self, query_image_path, top_k=10):
        query_img = cv2.imread(query_image_path, cv2.IMREAD_GRAYSCALE)
        query_features = self.calculate_hog_features(query_img)

        distances = euclidean_distances([query_features], self.features).flatten()
        sorted_indices = np.argsort(distances)[:top_k]

        return [
            (self.image_paths[i], os.path.basename(self.image_paths[i]))
            for i in sorted_indices
        ]

    def plot_results(self, query_image_path, result_image_info):
        fig, axes = plt.subplots(2, 5, figsize=(15, 6))

        # Plot query image
        query_img = cv2.imread(query_image_path, cv2.IMREAD_GRAYSCALE)
        axes[0, 0].imshow(query_img, cmap='gray')
        axes[0, 0].set_title("Query Image")
        axes[0, 0].axis("off")

        # Plot similar images
        for i, (result_image_path, result_image_name) in enumerate(result_image_info):
            img = cv2.imread(result_image_path, cv2.IMREAD_GRAYSCALE)
            axes[i // 5, i % 5].imshow(img, cmap='gray')
            axes[i // 5, i % 5].set_title(result_image_name)
            axes[i // 5, i % 5].axis("off")

        plt.tight_layout()
        plt.show()

    def calculate_euclidean_distances(self, query_image_path):
        query_img = cv2.imread(query_image_path, cv2.IMREAD_GRAYSCALE)
        query_features = self.calculate_hog_features(query_img)

        distances = euclidean_distances([query_features], self.features).flatten()
        return distances
    
    def evaluate_performance(self, query_images_path):
        all_true_labels = []
        all_distances = []
        precision_list = []
        recall_list = []
        f1_score_list = []
        
        # Load ground truth
        ground_truth_labels = [1 if 'horses' in path and '710.jpg' <= path.split(os.path.sep)[-1] <= '799.jpg' else 0 for path in self.image_paths]

        # Initialize arrays to store counts for each query
        TP_list = []
        FP_list = []
        TN_list = []
        FN_list = []

        # Loop through each query image
        for query_image in os.listdir(query_images_path):
            # Initialize counts for the current query
            TP = 0
            FP = 0
            TN = 0
            FN = 0

            query_image_path = os.path.join(query_images_path, query_image)
            distances = self.calculate_euclidean_distances(query_image_path)
            sorted_indices = np.argsort(distances)

            true_labels = ground_truth_labels  # Use ground truth labels for evaluation

            # Count the relevant images in the top-k results
            for i in range(10):
                if true_labels[sorted_indices[i]] == 1:
                    TP += 1
                else:
                    FP += 1

            # Count the non-relevant images after the top-k results
            for i in range(10, len(self.image_paths)):
                if true_labels[sorted_indices[i]] == 0:
                    TN += 1
                else:
                    FN += 1

            # Append counts for the current query to the arrays
            TP_list.append(TP)
            FP_list.append(FP)
            TN_list.append(TN)
            FN_list.append(FN)

            # Store results for averaging
            all_true_labels.extend(true_labels)
            all_distances.extend(distances)

            # Calculate metrics for the current query
            precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, distances <= distances[sorted_indices[9]], average='binary')

            # Store metrics for averaging
            precision_list.append(precision)
            recall_list.append(recall)
            f1_score_list.append(f1_score)

            print("Metrics for Query:", query_image)
            print("Precision:", precision)
            print("Recall:", recall)
            print("F1 Score:", f1_score)

        # Calculate and print average Precision, Recall, and F1 Score
        avg_precision = sum(precision_list) / len(precision_list)
        avg_recall = sum(recall_list) / len(recall_list)
        avg_f1_score = sum(f1_score_list) / len(f1_score_list)

        print("Average Precision:", avg_precision)
        print("Average Recall:", avg_recall)
        print("Average F1 Score:", avg_f1_score)
        
        # Manually calculate FPR and TPR
        sorted_indices_all = np.argsort(all_distances)
        total_positives = sum(all_true_labels)
        total_negatives = len(all_true_labels) - total_positives

        fpr = [0]
        tpr = [0]
        for i in range(len(all_true_labels)):
            if all_true_labels[sorted_indices_all[i]] == 1:
                tpr.append(tpr[-1] + 1 / total_positives)
                fpr.append(fpr[-1])
            else:
                tpr.append(tpr[-1])
                fpr.append(fpr[-1] + 1 / total_negatives)

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve with AUC = {:.2f}'.format(auc(fpr, tpr)))
        plt.legend(loc="lower right")
        plt.show()

All images were converted into `Grayscale` as they have a single channel representing intensity, making them computationally more efficient. They provide a uniform representation, focusing on structural details based on intensity variations. Grayscale images also reduce noise by eliminating the complexity of color information. They are effective at capturing object shapes and structures based on local gradient information. They are invariant to color changes, providing a consistent representation across diverse images. Grayscale images align with the HOG implementation, making them more effective for retrieval goals.


In [ ]:
def cbir_system_using_hog_features(dataset_path, query_images_path):
    cbir_system_hog = CBIRSystemHOG(dataset_path)
    cbir_system_hog.load_images()
    cbir_system_hog.extract_features()
    print("Number of images in the dataset: ", len(cbir_system_hog.image_paths))

    for query_image in os.listdir(query_images_path):
        query_image_path = os.path.join(query_images_path, query_image)

        plt.imshow(cv2.imread(query_image_path, cv2.IMREAD_GRAYSCALE), cmap='gray')
        plt.title("Query Image")
        plt.axis("off")
        plt.show()

        result_images = cbir_system_hog.search_similar_images(query_image_path, top_k=10)
        cbir_system_hog.plot_results(query_image_path, result_images)

    # Evaluate performance
    cbir_system_hog.evaluate_performance(query_images_path)

In [ ]:
measure_memory_usage_and_time_spent(cbir_system_using_hog_features, dataset_path, query_images_path)

## Discussion

HOG (Histogram of Oriented Gradients) may outperform Content-Based Image Retrieval (CBIR) with color histograms and color moments in certain scenarios due to its ability to capture shape and edge information, robustness to lighting and color variations, and discriminative power for specific tasks. HOG features capture the shapes, contours, and textures of an image through the distribution of oriented gradients, providing spatial information about the image. They are also relatively robust to lighting variations and color changes, making them effective for comparing images that might differ in brightness or overall color tone but share similar shapes and textures.

However, HOG features can be computationally more expensive to calculate compared to color histograms and color moments, especially when dealing with large datasets or real-time applications. They might not be the best choice for tasks where color is the primary distinguishing factor, such as identifying specific color variations in textiles or artwork.

The best choice for your CBIR system depends on your specific needs and dataset characteristics. HOG features may be a powerful option for retrieving images based on shapes, textures, and object recognition, but color-based features might be more suitable for tasks where color plays a dominant role in retrieval goals. Experimentation and evaluation are key to finding the optimal feature representation for your CBIR system.


### Conclusion

This study explores the role of color features in Content-Based Image Retrieval (CBIR) tasks, focusing on the development and evaluation of a CBIR system. The study found that color histograms, color moments, and Histogram of Oriented Gradients (HOG) features were effective in capturing nuanced color relationships. However, color histograms showed moderate performance, while color moments showed improved retrieval results due to their statistical representation. HOG features, which focused on shape and texture, achieved the highest retrieval accuracy, highlighting the importance of structural information for comprehensive image similarity assessment. The study also highlighted the need for a balance between accuracy, memory usage, and processing time in each feature set, highlighting the need for careful consideration of task and resource constraints when selecting the optimal feature.

### Effectiveness of Color Features

Color features play a crucial role in image similarity, but their effectiveness in Content-Based Image Retrieval (CBIR) depends on several factors. Color moments are more effective than color histograms, as they incorporate statistical information for richer image representations. HOG features are the most effective, capturing object shapes and structures. The method's ability to operate on grayscale images, focus on local gradient information, and provide robustness to color variations contributes to its success. The choice of features depends on the purpose of the CBIR system, which includes shape and texture information for broader recognition tasks.

### Insights and Recommendations

The findings underscore the importance of directing future research efforts towards innovative approaches in content-based image retrieval (CBIR). Hybrid feature representations, combining both color and shape-based features, present a promising avenue for enhancing retrieval performance. Emphasizing the alignment of task-specific features with the goals and characteristics of CBIR systems is crucial, urging researchers to tailor feature selections to the specific requirements of their datasets. Exploring advanced feature extraction techniques, including the integration of convolutional neural networks (CNNs) for automatic feature extraction and learning, holds potential for further improvements. The prospect of personalized CBIR systems, adapting retrieval results based on user preferences and search history, introduces a user-centric dimension to enhance the overall search experience. Furthermore, domain-specific optimization strategies, encompassing feature selection, distance metrics, and learning algorithms, can be explored to tailor CBIR solutions for specific application domains such as medical imagery, art retrieval, or product search. These insights collectively provide a roadmap for future research endeavors aimed at advancing the effectiveness and applicability of CBIR systems.

Finally, the study highlights the importance of feature representation in Computer-Based Image Retrieval (CBIR) tasks, highlighting the effectiveness of color-based and shape-based features. Color moments outperformed color histograms, while HOG features are effective for capturing intricate object details. The findings suggest that selecting features aligned with task requirements can improve efficiency and user-friendliness in CBIR systems. Understanding image characteristics, retrieval goals, and feature combinations is crucial for unlocking color's full potential in CBIR systems.